In [1]:
from IPython.display import Markdown, display

from dotenv import load_dotenv
import os

load_dotenv()

True

# Search

## Genereate Embeddings

In [2]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
import os


# Variables not used here do not need to be updated in your .env file
endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
credential = AzureKeyCredential(os.environ["AZURE_SEARCH_ADMIN_KEY"]) if len(os.environ["AZURE_SEARCH_ADMIN_KEY"]) > 0 else DefaultAzureCredential()
index_name = os.environ["AZURE_SEARCH_INDEX"]

In [3]:
from openai import OpenAI
from openai import embeddings


# Set the API key and endpoint
api_key = os.getenv('OPENAI_API_KEY')
org = os.getenv('OPENAI_ORG')
project = os.getenv('OPENAI_PROJECT')
# model = "gpt-5-mini-2025-08-07"
embedding_model = "text-embedding-3-small"



In [6]:

def generate_embeddings(text):
    client = OpenAI(
        api_key=api_key,
        organization=org,
        project=project
    )

    embedding = client.embeddings.create(
        model=embedding_model,
        input=text
    )
    embedding = embedding.data[0].embedding
    return embedding

print(generate_embeddings("Hello, world!")[:10])

[-0.019143931567668915, -0.025292053818702698, -0.0017211713129654527, 0.01883450709283352, -0.03382139280438423, -0.019682060927152634, -0.02102738246321678, 0.05160655081272125, -0.03218010067939758, -0.03043118305504322]


## query



In [7]:
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery, VectorQuery, VectorizedQuery, QueryType

# Pure Vector Search
query = "eye test"
# "Which is more comprehensive, Northwind Health Plus vs Northwind Standard?"

topk = 5

search_client = SearchClient(endpoint, index_name, credential=credential)


In [8]:
[qt for qt in QueryType]

[<QueryType.SIMPLE: 'simple'>,
 <QueryType.FULL: 'full'>,
 <QueryType.SEMANTIC: 'semantic'>]

In [9]:


# vector_fields = ["content_vector"]

# vector_queries = [VectorizableTextQuery(
#         text = query,
#         k_nearest_neighbors=topk,
#         fields=vector,
#         exhaustive=True,
#     ) for vector in vector_fields]


# vector_queries = [VectorQuery(
#     k_nearest_neighbors=topk,
#     fields="content_vector",
#     exhaustive=True,
# )]

# https://github.com/Azure/azure-search-vector-samples/blob/main/demo-python/code/basic-vector-workflow/azure-search-vector-python-sample.ipynb
# https://learn.microsoft.com/en-us/python/api/azure-search-documents/azure.search.documents.models.vectorizedquery?view=azure-python
vector_query = VectorizedQuery(
    vector=generate_embeddings(query),
    k_nearest_neighbors=topk,
    fields="content_vector",
    exhaustive=True,

)


# vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=topk, fields="vector", exhaustive=True)
# Use the below query to pass in the raw vector query instead of the query vectorization
# vector_query = RawVectorQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="vector")

# VectorizedQuery Class
# The query parameters to use for vector search when a raw vector value is provided.
# VectorizableTextQuery Class
# The query parameters to use for vector search when a text value that needs to be vectorized is provided.

# results = search_client.search(
#     search_text=None,
#     vector_queries= vector_queries,
#     select=["parent_id", "chunk_id", "chunk","title"],
#     top=topk
# )

# check basic vector workflow sample: https://github.com/Azure/azure-search-vector-samples/blob/main/demo-python/code/basic-vector-workflow/azure-search-vector-python-sample.ipynb
# when providing search_text, this bcomes a hybrid search
# adding semantic_configuration_name enables semantic ranking of the results
# query rewriting is an additional feature that can be enabled
results = search_client.search(
        search_text = "PSI1454426",
        vector_queries=[VectorizedQuery(
            vector=generate_embeddings(query),
            k_nearest_neighbors=topk,
            fields="content_vector",
            exhaustive=True,

        )],
        select=["chunk_id", "file_name", "base_name", "content", "content_vector"],
        top=topk,
        include_total_count=True,
        # query_type=QueryType.FULL,
        semantic_configuration_name="my-semantic-config",

    )




for result in results:
    print(f"chunk_id: {result['chunk_id']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['content']}")


    # print(f"Content: {result['chunk']}")


chunk_id: dometic_fridge_stove_Sept_2025_20250918_114057_0001
Score: 0.01666666753590107
Content: # TAX INVOICE  
<table>
<tr>
<th colspan="2">Invoice No. PSI1454426</th>
</tr>
<tr>
<th>PAGE</th>
<th>DATE</th>
</tr>
<tr>
<td>1</td>
<td>16/09/25</td>
</tr>
</table>  
$ 1,613.01
\* Due and Payable by
16 September 2025  
Dometic Australia Pty Ltd
ABN: 62086366305
7 Nursery Ave
CLAYTON SOUTH VIC 3169
Australia
Tel: 1300 304 220  
ACCOUNT NAME:
Mutaz Abu Ghazaleh - +61 447 663 887
VARSITY LAKES QLD 4227
Australia  
DELIVER TO:  
Mutaz Abu Ghazaleh - +61 447 663 887  
3 Jack McNamee Place  
KELLYVILLE NSW 2155  
Australia  
<table>
<tr>
<th>CUSTOMER NO.</th>
<th>OUR REF.</th>
<th>CUSTOMER REF.</th>
<th>ORDER DATE</th>
<th>INPUT</th>
<th>SENT BY</th>
</tr>
<tr>
<td>9454</td>
<td>SO3318449</td>
<td>2392S</td>
<td>15/09/25</td>
<td>MAT.GRAH</td>
<td>BORDER EXPRESS</td>
</tr>
</table>  
<table>
<tr>
<th>QTY</th>
<th>ITEM CODE</th>
<th>DESCRIPTION</th>
<th>UNIT PRICE</th>
<th>UNIT</th>
<th>DISC %

In [10]:
# define a search function

def search_query(query_text, top_k=5, vector_only=False):
    vector = generate_embeddings(query_text)
    vector_query = VectorizedQuery(
        vector=vector,
        k_nearest_neighbors=top_k,
        fields="content_vector",
        exhaustive=True,
    )

    search_text = None if vector_only else query_text
    results = search_client.search(
        search_text=search_text,
        vector_queries=[vector_query],
        select=["chunk_id", "file_name", "base_name", "content", "content_vector"],
        top=top_k,
        include_total_count=True,
        # query_type=QueryType.FULL,
        semantic_configuration_name="my-semantic-config",
    )
    return list(results)

In [11]:
def search_display_result(query_text, vector_only=False):
    results  = search_query(query_text, vector_only=vector_only)
    for i, result in enumerate(results):
        display(Markdown(f"### Result {i+1}"))
        display(Markdown(f"**Chunk ID:** {result['chunk_id']}"))
        display(Markdown(f"**Score:** {result['@search.score']}"))
        display(Markdown(f"**Content:** {result['content']}"))
        display(Markdown("---"))

In [12]:
search_display_result("3 Jack Mcnamee Place")

### Result 1

**Chunk ID:** dometic_fridge_stove_Sept_2025_20250918_114057_0001

**Score:** 0.03333333507180214

**Content:** # TAX INVOICE  
<table>
<tr>
<th colspan="2">Invoice No. PSI1454426</th>
</tr>
<tr>
<th>PAGE</th>
<th>DATE</th>
</tr>
<tr>
<td>1</td>
<td>16/09/25</td>
</tr>
</table>  
$ 1,613.01
\* Due and Payable by
16 September 2025  
Dometic Australia Pty Ltd
ABN: 62086366305
7 Nursery Ave
CLAYTON SOUTH VIC 3169
Australia
Tel: 1300 304 220  
ACCOUNT NAME:
Mutaz Abu Ghazaleh - +61 447 663 887
VARSITY LAKES QLD 4227
Australia  
DELIVER TO:  
Mutaz Abu Ghazaleh - +61 447 663 887  
3 Jack McNamee Place  
KELLYVILLE NSW 2155  
Australia  
<table>
<tr>
<th>CUSTOMER NO.</th>
<th>OUR REF.</th>
<th>CUSTOMER REF.</th>
<th>ORDER DATE</th>
<th>INPUT</th>
<th>SENT BY</th>
</tr>
<tr>
<td>9454</td>
<td>SO3318449</td>
<td>2392S</td>
<td>15/09/25</td>
<td>MAT.GRAH</td>
<td>BORDER EXPRESS</td>
</tr>
</table>  
<table>
<tr>
<th>QTY</th>
<th>ITEM CODE</th>
<th>DESCRIPTION</th>
<th>UNIT PRICE</th>
<th>UNIT</th>
<th>DISC %</th>
<th>LINE TOTAL EXCL. GST</th>
<th>LOC</th>
</tr>
<tr>
<td>1</td>
<td>9620001868</td>
<td>REF/FZR 1DOOR COMP ACDC NRX80 Serial No: 51800028</td>
<td>$ 1,177.27</td>
<td>Each</td>
<td></td>
<td>$ 1,177.27</td>
<td>1CL</td>
</tr>
<tr>
<td>1</td>
<td>9610004436</td>
<td>2 COOK 3 PRO DELUXE</td>
<td>$ 244.55</td>
<td>Each</td>
<td></td>
<td>$ 244.55</td>
<td>1CL</td>
</tr>
<tr>
<td>1</td>
<td>9630000028</td>
<td>NRX BOTTOM HANDLE ASSEMBLY</td>
<td>$ 44.55</td>
<td>Each</td>
<td></td>
<td>$ 44.55</td>
<td>1CL</td>
</tr>
<tr>
<td></td>
<td></td>
<td>Serial No: 52800165</td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td></td>
<td></td>
<td>Sydney 4×4 mutazag@hotmail.com</td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
</table>  
SPECIAL DELIVERY INSTRUCTIONS:  
TERMS:  
CBD  
SHIPMENT NO:
DMSS1359770  
SUB TOTAL  
$ 1,466.37  
GST (10%)  
$ 146.64  
<table>
<tr>
<th>CARTONS</th>
<th>KG</th>
<th>CUBIC M</th>
<th>TOTAL INCL. GST</th>
<th>$ 1,613.01</th>
</tr>
<tr>

---

...

### Result 5

**Chunk ID:** Northwind_Health_Plus_Benefits_Details_0131

**Score:** 0.016129031777381897

**Content:** department.  
. You may also want to contact a legal professional to help you with the appeals process.  
· If Northwind Health Plus denies your appeal, you may be able to file a lawsuit against
them.  
· If Northwind Health Plus denies your appeal, you may be able to file a complaint with the
California Department of Managed Health Care.  
Understanding the appeals process can help you get the coverage and care you need. It is
important to remember that you have the right to appeal a decision made by Northwind
Health Plus or your provider. It is also important to remember that you must file your
appeal within 60 days of the date of the denial letter or other written notification from
Northwind Health Plus, unless Northwind Health Plus has given you more time. You should
also include all relevant medical and other information with your appeal and contact a legal
professional if you need help.  
Appeal Levels  
COMPLAINTS AND APPEALS  
At Northwind Health Plus, we take complaints and appeals seriously, and we strive to
provide our members with the highest quality of care. We have a multi-level process in
place to ensure that any grievances are addressed and resolved quickly and fairly.  
Level 1: Initial Review  
This is the first step in the appeals process. If you have a problem with a service or product
received, contact the plan's Customer Service team. The team will review your complaint to
determine if a solution can be reached or if the complaint should be escalated to the next
level.  
Level 2: Formal Complaint  
If a satisfactory resolution is not reached at Level 1, you may submit a formal written
complaint to Northwind Health Plus. This should include details of the issue, the date and
time of the incident, and the names of any involved personnel. It should also include any
relevant documentation or other evidence that supports your claim.  
Level 3: External Review  
<!-- PageBreak -->

---

In [13]:
search_display_result("Role of a CEO")

### Result 1

**Chunk ID:** employee_handbook_0017

**Score:** 0.03306011110544205

**Content:** ## Job Roles  
1\. Chief Executive Officer  
2\. Chief Operating Officer  
3\. Chief Financial Officer  
4\. Chief Technology Officer  
5\. Vice President of Sales  
6\. Vice President of Marketing  
7\. Vice President of Operations  
8\. Vice President of Human Resources  
9\. Vice President of Research and Development  
10\. Vice President of Product Management  
11\. Director of Sales  
12\. Director of Marketing  
13\. Director of Operations  
14\. Director of Human Resources  
<!-- PageBreak -->  
15\. Director of Research and Development  
16\. Director of Product Management  
17\. Senior Manager of Sales  
18\. Senior Manager of Marketing
19\. Senior Manager of Operations  
20\. Senior Manager of Human Resources  
21\. Senior Manager of Research and Development  
22\. Senior Manager of Product Management  
23\. Manager of Sales  
24\. Manager of Marketing
25\. Manager of Operations  
26\. Manager of Human Resources
27\. Manager of Research and Development  
28\. Manager of Product Management  
29\. Sales Representative  
30\. Customer Service Representative

---

...

### Result 5

**Chunk ID:** Northwind_Health_Plus_Benefits_Details_0114

**Score:** 0.016129031777381897

**Content:** ### Personal Health Support Programs CARE MANAGEMENT  
Northwind Health Plus offers a number of personal health support programs to help
members stay healthy and manage their health care costs. Through this program, members
can access a range of services, programs, and benefits including:  
Case Management: Northwind Health Plus provides a case management program that
connects members with a team of health professionals, depending on the individual's needs.
These professionals will help assess the member's health situation, develop a plan of care,
coordinate care and resources, and provide support and education.  
Disease Management: Northwind Health Plus offers disease management programs for
members with certain chronic conditions. These programs provide members with support,
information, and resources about their conditions, as well as assistance in managing their
health care.  
Wellness Programs: Northwind Health Plus provides wellness programs to help members
stay healthy and manage their health care costs. These programs include programs to help
members quit smoking, manage stress, and improve their overall health and well-being.  
Exceptions:  
· Members must be enrolled in Northwind Health Plus to be eligible for these programs.  
.These programs may not be available in all areas.  
.Some services may not be covered by Northwind Health Plus.  
Tips:  
·Take advantage of the services and programs offered through Northwind Health Plus.  
.Talk to your doctor or other health care provider about your health and any treatments
that may be available.  
.Take an active role in your health care. Ask questions and be informed about your health
and any treatments that may be available.  
. Make sure to follow your doctor's instructions and stay up to date on your health care.  
. If you have any questions or concerns about your health, contact Northwind Health Plus
for assistance.  
<!-- PageBreak -->

---

In [18]:
search_display_result("PSI14554426")

### Result 1

**Chunk ID:** dometic_fridge_stove_Sept_2025_20250918_114057_0001

**Score:** 0.01666666753590107

**Content:** # TAX INVOICE  
<table>
<tr>
<th colspan="2">Invoice No. PSI1454426</th>
</tr>
<tr>
<th>PAGE</th>
<th>DATE</th>
</tr>
<tr>
<td>1</td>
<td>16/09/25</td>
</tr>
</table>  
$ 1,613.01
\* Due and Payable by
16 September 2025  
Dometic Australia Pty Ltd
ABN: 62086366305
7 Nursery Ave
CLAYTON SOUTH VIC 3169
Australia
Tel: 1300 304 220  
ACCOUNT NAME:
Mutaz Abu Ghazaleh - +61 447 663 887
VARSITY LAKES QLD 4227
Australia  
DELIVER TO:  
Mutaz Abu Ghazaleh - +61 447 663 887  
3 Jack McNamee Place  
KELLYVILLE NSW 2155  
Australia  
<table>
<tr>
<th>CUSTOMER NO.</th>
<th>OUR REF.</th>
<th>CUSTOMER REF.</th>
<th>ORDER DATE</th>
<th>INPUT</th>
<th>SENT BY</th>
</tr>
<tr>
<td>9454</td>
<td>SO3318449</td>
<td>2392S</td>
<td>15/09/25</td>
<td>MAT.GRAH</td>
<td>BORDER EXPRESS</td>
</tr>
</table>  
<table>
<tr>
<th>QTY</th>
<th>ITEM CODE</th>
<th>DESCRIPTION</th>
<th>UNIT PRICE</th>
<th>UNIT</th>
<th>DISC %</th>
<th>LINE TOTAL EXCL. GST</th>
<th>LOC</th>
</tr>
<tr>
<td>1</td>
<td>9620001868</td>
<td>REF/FZR 1DOOR COMP ACDC NRX80 Serial No: 51800028</td>
<td>$ 1,177.27</td>
<td>Each</td>
<td></td>
<td>$ 1,177.27</td>
<td>1CL</td>
</tr>
<tr>
<td>1</td>
<td>9610004436</td>
<td>2 COOK 3 PRO DELUXE</td>
<td>$ 244.55</td>
<td>Each</td>
<td></td>
<td>$ 244.55</td>
<td>1CL</td>
</tr>
<tr>
<td>1</td>
<td>9630000028</td>
<td>NRX BOTTOM HANDLE ASSEMBLY</td>
<td>$ 44.55</td>
<td>Each</td>
<td></td>
<td>$ 44.55</td>
<td>1CL</td>
</tr>
<tr>
<td></td>
<td></td>
<td>Serial No: 52800165</td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td></td>
<td></td>
<td>Sydney 4×4 mutazag@hotmail.com</td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
</table>  
SPECIAL DELIVERY INSTRUCTIONS:  
TERMS:  
CBD  
SHIPMENT NO:
DMSS1359770  
SUB TOTAL  
$ 1,466.37  
GST (10%)  
$ 146.64  
<table>
<tr>
<th>CARTONS</th>
<th>KG</th>
<th>CUBIC M</th>
<th>TOTAL INCL. GST</th>
<th>$ 1,613.01</th>
</tr>
<tr>

---

...

### Result 5

**Chunk ID:** Northwind_Standard_Benefits_Details_0000

**Score:** 0.015625

**Content:** # Contoso Electronics  
Northwind Health Standard Plan  
<figure>  
Contoso
Electronics  
</figure>  
<!-- PageBreak -->  
This document contains information generated using a language model (Azure OpenAI). The
information contained in this document is only for demonstration purposes and does not
reflect the opinions or beliefs of Microsoft. Microsoft makes no representations or
warranties of any kind, express or implied, about the completeness, accuracy, reliability,
suitability or availability with respect to the information contained in this document.  
All rights reserved to Microsoft  
<!-- PageBreak -->

---

In [ ]:
search_display_result("PSI14554426", vector_only=True)